# Annual Sea Level Anomalies
```{glue:figure} SL_MMA_annual_station
:scale: 25%
:align: right
```

In this notebook, we'll be taking a look at sea level anomalies (SLA), which can be thought of as signals that differ from the climatology (of a station or region). For these products we'll focus on yearly anomalies, and leave the monthly anomalies (for the most part) to the [inter-annual notebook](notebooks/regional_and_local/SL_anomaly_annual.ipynb). For a nice overview and animation of daily mean SLAs in the Hawaii and eastern Pacific region, see [Pacific Sea Level Monitoring](https://uhslc.soest.hawaii.edu/pacific-sea-level-monitoring/).

Download Files:
[Map](https://uhslc.soest.hawaii.edu/jfiedler/SeaLevelIndicators/output/Hawaii_Region_Output/1.2_2_SL_anomaly_annual_map_decadal.png) |
[Time Series Plot](https://uhslc.soest.hawaii.edu/jfiedler/SeaLevelIndicators/output/Hawaii_Region_Output/1.2_SL_MMA_annual.png)

## Setup

As with previous sections, we first need to import the necessary libraries, establish our input/output directories, and set up some basic plotting rules. We'll do this by running another notebook called "setup" and "plotting functions," and then we'll set our data and output paths. If you have not run the datawrangling notebook, you will need to do this before proceeding here.


In [ ]:
%run ../setup.ipynb
from plotting_functions import plot_map, plot_zebra_frame

# check to make sure that data_dir/rsl_daily_hawaii.nc exists, if not, make warning to run datawrangling notebook
if not (data_dir / 'rsl_daily_hawaii.nc').exists():
    print('rsl_daily_hawaii.nc not found in data directory. Please run the data wrangling notebook first')
else:
    print('rsl_daily_hawaii.nc found in data directory. Proceed.')

### Import the tide gauge data

In [ ]:
#import rsl_daily
rsl_daily = xr.open_dataset(data_dir/ 'rsl_daily_hawaii.nc')

### Do some data wrangling again. 
#### 1. Check the data and drop all stations with less than 80% coverage

In [ ]:
data_coverage = rsl_daily.rsl_mhhw.count(dim='time')/len(rsl_daily.time)

#drop all locations with data_coverage less than 80%
rsl_subset = rsl_daily.where(data_coverage>0.80,drop=True)
rsl_daily.close()

rsl_subset

#### 2. Now we'll remove the trend (over the entire record) from this subset of stations.

In [ ]:
trend_mag, sea_level_trend, trend_rate = tsf.process_trend_with_nan(rsl_subset['rsl_anomaly'])

rsl_subset['rsl_anomaly_detrended'] = rsl_subset['rsl_anomaly'] - sea_level_trend

#### 3. Resample to monthly time series of anomalies.

In [ ]:
# resample to monthly time series 
rsl_monthly = rsl_subset.resample(time='1MS',label='left').mean()

# get climatology (annual cycle) by grouping by month
rsl_climatology = rsl_monthly.groupby('time.month').mean('time')

# get anomalies by subtracting climatology from monthly data
rsl_anomalies = rsl_monthly.groupby('time.month') - rsl_climatology

rsl_anomalies

#### 4. Now sort the subset on "storm year" instead of calendar year for yearly means.

In [ ]:
# change the index to be on storm_time instead of time
rsl_subset = rsl_subset.set_index(time='storm_time')

# rename the time dimension to storm_time
rsl_subset = rsl_subset.rename({'time':'storm_time'})

rsl_subset

In [ ]:
# reorder to that storm_time is monotonicly increasing
rsl_subset = rsl_subset.sortby('storm_time')

# Group by storm_year and record_id
rsl_yearly_mean = rsl_subset.groupby('storm_time.year').mean('storm_time')

# give year dimension a name
rsl_yearly_mean = rsl_yearly_mean.rename({'year':'storm_year'})

rsl_yearly_mean

## Absolute Value: satellite (trend removed)

In [ ]:
# open the cmems data
cmems_big = xr.open_dataset(data_dir / 'cmems_L4_SSH_0.125deg_1993_2025hawaii.nc')
cmems = cmems_big.sel(longitude=slice(180,208), latitude=slice(14.5, 30.5)) # we'll cut this down to a reasonable slice for analysis and plotting. 

In [ ]:
cmems

### Plot an area-wide map, with stations
First we'll need to make sure that our data is detrended properly in order to look at yearly anomalies.

In [ ]:
# concentrating on decadal trends
# let's look at 1993-2003 and 2003-2013

trend_mag, sea_level_trend, trend_rate = tsf.process_trend_with_nan(cmems['sla'].sel(time=slice('1993','2023')),weighted=False)

In [ ]:
# define vmin and vmax variables
vmin = 0
vmax = 0.5
# Use float values for map extent to avoid dtype issues and ensure gridlines appear
xlims = [float(cmems.longitude.min()), float(cmems.longitude.max())]
ylims = [float(cmems.latitude.min()), float(cmems.latitude.max())]
trend_rate_cm = trend_rate * 100  # convert to cm/yr

# make ax,fig
crs = ccrs.PlateCarree()
cmap = 'YlOrRd'
fig, ax, crsMap = plot_map(vmin,vmax,cmap,xlims,ylims)

trend_rate_cm.plot(ax=ax, transform=crs,
                         vmin=vmin, vmax=vmax, cmap=cmap, add_colorbar=True, 
                         cbar_kwargs={'label': 'Sea Level Trend rate (cm/yr, 1993-2024)'}
)

# Call the function to add the zebra frame
plot_zebra_frame(ax, lw=5, segment_length=2, crs=crsMap)

# save the figure
plt.savefig(output_dir / '1.2_cmems_trend_rate_1993_2024.png', dpi=300, bbox_inches='tight')


### Area-weighted average trend 

As an example, we can use the entire grid for this. That said, in reality, you'd want to pick boundaries that make more sense. You could, for example, use the EEZ.

In [ ]:
startYear = '1993'
endYear = '2024'
trend_mag, sea_level_trend, trend_rate = tsf.process_trend_with_nan(cmems['sla'].sel(time=slice(startYear,endYear)),weighted=True)

print(f'The trend rate for {startYear}-{endYear} in the entire grid is {trend_rate*100:.2f} cm/yr')


In [ ]:
# concentrating on decadal trends
# let's look at 1993-2003 and 2003-2013
cmems1993_2003 = cmems.sel(time=slice('1993','2003'))

# get the overall trend (full time series)
trend_mag, sea_level_trend, trend_rate = tsf.process_trend_with_nan(cmems['sla'])
trend_mag_area_weighted, sea_level_trend_area_weighted, trend_rate_area_weighted = tsf.process_trend_with_nan(cmems['sla'],weighted=True)


#remove the trend from the data
sla_detrended = cmems1993_2003['sla'] - sea_level_trend.sel(time=slice('1993','2003'))
sla_detrended_area_weighted = cmems1993_2003['sla'] - sea_level_trend_area_weighted.sel(time=slice('1993','2003'))

Let's check this for a random station in our list.

In [ ]:
id = 3
lat = rsl_subset.isel(station_id=id).lat
lon = rsl_subset.isel(station_id=id).lon
sla_detrended.sel(latitude=lat, longitude=lon, method='nearest').plot(label='Sea Level Anomaly (m)')
sla_detrended_area_weighted.sel(latitude=lat, longitude=lon, method='nearest').plot(label='Area Weighted Sea Level Anomaly (m)')

# change the title to station name
plt.title(rsl_subset['station_name'].isel(station_id=id).values)
#add axis labels
plt.xlabel('Year')
plt.ylabel('Sea Level Anomaly (m)')

plt.legend()

There are slight differences between removing the grid-level trend versus the area-weighted trend in this plot. Nothing too major here.

### Create a map
Concentrating on decadal trends.

In [ ]:
# concentrating on decadal trends
# let's look at 1993-2003 and 2003-2013

yr_start = [1993,2003,2013,2003]
yr_stop = [2003,2013,2023,2023]

#do storm year instead of calendar year

yr_start_str = [str(int(yr))+'-05-01' for yr in yr_start]
yr_stop_str = [str(int(yr))+'-04-30' for yr in yr_stop]

trend_mag, sea_level_trend, trend_rate = tsf.process_trend_with_nan(cmems['sla'].sel(time=slice(yr_start_str[0],yr_stop_str[-1])),weighted=False)

#remove the trend from the data
sla_detrended = cmems['sla'] - sea_level_trend


# make 4 subplots for each year range
from matplotlib.ticker import FuncFormatter

def pacific_all_west_formatter(x, pos=None):
    # Convert to [0, 360)
    x = (x + 360) % 360
    deg_west = (180 - x) % 360
    if deg_west == 0:
        return "180°W"
    else:
        return f"{abs(deg_west):.0f}°W"

crs_main = ccrs.PlateCarree(central_longitude=180)
crs_sub = ccrs.PlateCarree()
fig, ax = plt.subplots(2,2,figsize=(8, 5), subplot_kw={'projection': crs_main})

# Variable to hold the colorbar handle
cbar_ax = fig.add_axes([0.92, 0.15, 0.03, 0.7])  # Adjust these values to fit your layout
fig.subplots_adjust(right=0.9)  # Make space for the colorbar
 
vmin = -0.03
vmax = 0.03
# Loop through each period
for i, ax in enumerate(ax.flat):
    # Select the data for the current period and calculate the mean over time
    sla_mean = sla_detrended.sel(time=slice(yr_start_str[i],yr_stop_str[i])).mean(dim='time',skipna=True)
    # sla_mean = sla_mean - sea_level_trend.sel(time=slice(str(yr_start[i]), str(yr_stop[i]))).mean(dim='time')

    #subtract the
    
    # Plot the mean sea level anomaly
    c = sla_mean.plot(ax=ax, add_colorbar=False, transform=crs_sub, cmap='RdBu_r')
    
    # Set the title to the current period
    ax.set_title(f'{yr_start[i]}-{yr_stop[i]}')

    # Add coastlines and land  
    palette = sns.color_palette("RdBu_r", as_cmap=True)
    cmap = palette

    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.LAND, color='lightgrey')
    
    # Optionally, adjust color limits for consistency across plots
    c.set_clim(vmin,vmax)

    ax.scatter(rsl_yearly_mean['lon'], rsl_yearly_mean['lat'], transform=crs_sub, s=50,
              c='white', linewidth=0.5, edgecolor='black',zorder=10)

    ax.scatter(rsl_yearly_mean['lon'], rsl_yearly_mean['lat'], transform=crs_sub, s=100,
           c=rsl_anomalies.sel(time=slice(yr_start_str[i],yr_stop_str[i])).rsl_anomaly_detrended.mean(dim="time"), vmin=vmin, vmax=vmax, cmap=cmap,
           linewidth=0.5, edgecolor='black',zorder=10)
    
    #add grid
    gl = ax.gridlines(crs=crs_main, draw_labels=False, linestyle=':', color='black',
                      alpha=0.5,xlocs=ax.get_xticks(),ylocs=ax.get_yticks())
    gl.xformatter = FuncFormatter(pacific_all_west_formatter)
    
    if i>1:
        gl.bottom_labels = True

    if i==0:
        gl.left_labels = True

    if i==2:
        gl.left_labels = True

    #make all labels tiny
    gl.xlabel_style = {'size': 8}
    gl.ylabel_style = {'size': 8}

# Add the colorbar
cbar = fig.colorbar(c, cax=cbar_ax)
cbar.set_label('Sea Level Anomaly (m)')

glue("SL_YMA_decadal",fig,display=False)

plt.savefig(output_dir / '1.2_2_SL_anomaly_annual_map_decadal.png', dpi=300, bbox_inches='tight')


```{glue:figure} SL_YMA_decadal
:name: "SL_YMA_decadal"

Decadal (or bidecadal) mean anomalies of the Hawaiian Islands region, background trend (Storm Years 1993-2023) removed. Note that the sea level is plotted in units of m. The mean anomaly is near zero when using the full timeseries and grid-level trends (i.e. no area-weighted averaging). Mean monthly anomalies at the stations are plotted as well, showing similar-ish patterns.
```

## Time Series Plots
Now we'll look at monthly and yearly anomalies at the tide stations through time. We'll also plot ENSO signals on the same plot, to get a visual feel for any ENSO-driven anomalous effects on sea level.

In [ ]:
def plot_TG_rsl_anomaly_annual(rid, fig, ax, rsl_yearly_mean, rsl_subset, rsl_anomalies,oni, oniPlot=False):

    ax.set_xlim(1993,2024)
    ax.set_ylim(-0.3, 0.5)
    ax.grid(alpha=0.2, color='lightgray')

    if oniPlot:
        ymin, ymax = ax.get_ylim()
        # Shading El Niño events
        ax.fill_between(oni.index, ymin,ymax, 
                        where=oni['El Nino'] == 1, color='red', alpha=0.1, label='El Niño')
        # Shading La Niña events
        ax.fill_between(oni.index,  ymin,ymax,
                        where=oni['La Nina'] == 1, color='blue', alpha=0.1, label='La Niña')
    
    # get min and max for each year
    rsl_yearly_min = rsl_subset.groupby('storm_time.year').min('storm_time')
    rsl_yearly_max = rsl_subset.groupby('storm_time.year').max('storm_time')

    #make a fill from max to min for each month
    ax.fill_between(rsl_yearly_mean.storm_year+0.5, 
                     rsl_yearly_min['rsl_anomaly_detrended'].isel(station_id=rid), 
                     rsl_yearly_max['rsl_anomaly_detrended'].isel(station_id=rid), 
                     alpha=0.2, label='Range (Annual SLA Min-Max)')

    ax.plot(rsl_yearly_mean.storm_year+0.5, 
            rsl_yearly_mean['rsl_anomaly_detrended'].isel(station_id=rid), 
            label='Annual Mean Sea Level Anomaly (m)',alpha=1,linewidth=2)

    # convert rsl_monthly['time'] to float
    storm_time_float = rsl_monthly['time.year'] + (rsl_monthly['time.month']-5)/12 #subtract 5 months to go with 'storm year' convention (May-April)
    ax.plot(storm_time_float, 
            rsl_anomalies['rsl_anomaly_detrended'].isel(station_id=rid), 
            label='Monthly Mean Sea Level Anomaly (m)',alpha=1, color='gray',linewidth=0.75)
    ax.set_ylabel(''), ax.set_xlabel('')

    # add rsl_yearly_mean['station_name'].isel(station_id=rid).values to upper center of plot
    nameID = rsl_yearly_mean['station_name'].isel(station_id=rid).values + ' (' + rsl_yearly_mean['station_id'].isel(station_id=rid).values + ')'
    ax.text(0.5, 0.9, nameID, horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)


First, we'll plot one gauge

In [ ]:
fig,ax = plt.subplots(figsize=(10,5))
# add ENSO to the plots
# import oni.csv
oni = pd.read_csv(data_dir/'climate_indices/oni_bool.csv',header=0)
# change oni index to Date column
oni.index = pd.to_datetime(oni['Date'] )
#adjust the index to be float value of fractional year
oni.index = oni.index.year + (oni.index.month - 1)/12 - 5/12

sid = 1
plot_TG_rsl_anomaly_annual(sid, fig, ax,  rsl_yearly_mean, rsl_subset, rsl_anomalies,oni,oniPlot=True)
ax.legend(loc='upper right', ncol=1, bbox_to_anchor=(1.35, 1.0))

# Save the plot to a file
filename = '1.2_SL_MMA_annual_' + str(rsl_yearly_mean['station_name'].isel(station_id=sid).values) + '.png'
plt.savefig(output_dir / filename, dpi=300, bbox_inches='tight')

fig.text(0.05, 0.5, 'Sea Level Anomaly (m)', va='center', rotation='vertical')
fig.text(0.5, 0, 'Storm Year', ha='center')

glue('SL_MMA_annual_station',fig,display=False)
station = str(rsl_yearly_mean['station_name'].isel(station_id=sid).values)

glue('station',station,display=False)

```{glue:figure} SL_MMA_annual_station
:name: "SL_MMA_annual_station"

Yearly mean anomalies at the {glue:text}`station` tide gauge with background trend removed (blue). The shaded region about the annual mean denotes the annual means of the daily min and max sea level. Note that the sea level is plotted in units of m. The monthly mean anomaly (gray) is calculated with respect to the annual cycle. El Niño and La Niño time periods are derived from the Oceanic Niño Index (ONI), available at https://origin.cpc.ncep.noaa.gov/products/analysis_monitoring/ensostuff/ONI_v5.php  
```

### Create a time series plot
Now we will plot all the gauges.

In [ ]:
# get axis for the plots
fig,axes = plt.subplots(4,2,figsize=(15,10), sharex=True, sharey=True)
axes = axes.flatten()



station_ids = rsl_monthly.station_id.values
# iterate over each axis and plot the monthly mean
for i in range(len(station_ids)):
    ax = axes[i]
    plot_TG_rsl_anomaly_annual(i, fig, ax,  rsl_yearly_mean, rsl_subset, rsl_anomalies,oni,oniPlot=True)

    if i < 7:
        ax.set_xlabel('')    


#add El Nino and La Nina years


# axes[7].legend(loc='lower center', ncol=3,bbox_to_anchor=(-0.1, -1),fontsize=12)

# add y label
fig.text(0.06, 0.5, 'Sea Level Anomaly (m)', va='center', rotation='vertical')
fig.text(0.5, 0.06, 'Storm Year', ha='center')


# Save the plot to a file
plt.savefig(output_dir / '1.2_SL_MMA_annual.png', dpi=300, bbox_inches='tight')

glue("SL_MMA_annual",fig,display=False)



```{glue:figure} SL_MMA_annual
:name: "SL_MMA_annual"

Yearly mean anomalies at the each tide gauge with background trend removed (blue). The blue shaded region denotes the yearly means of the daily min and max sea level anomalies. Note that the sea level is plotted in units of m, with the epoch daily mean and trend removed. The monthly mean anomaly (gray) is calculated with respect to the annual cycle. 
```